# Catatan yang Diulik

1. Modality Based, mempertimbangkan kolom rating, jumlah reviews, nama tempat, tipe wisata, Waktu buka, dan koordinat
2. Interkoneksi antar tempat wisata, dengan mempertimbangkan nilai di user. Nilai yang dimaksud misal user A pernah merating berapa di tempat A, lalu telusuri dia pernah merating dimana saja. Nanti dijadikan identitas user yang digunakan untuk rekomendasi item, tapi soft recom, tidak user based.

Catatan:
- Untuk koordinat bisa coba ulik rumus **Haversine**
- Poin ke 2, bisa pake konsep inputan baru ke model rekomendasi, jadi inputan nnti di cross calculation dari hasil rekomendasi dari poin 1.
- Waktu membuka website atau waktu komputer dijadikan pertimbangan rekomendasi dari nilai waktu buka di data item.

In [29]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from math import radians, sin, cos, sqrt, atan2
from datetime import datetime
from IPython.display import display

fixedDf = pd.read_csv('data/fixedData.csv')
fixedDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455 entries, 0 to 454
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   place_id               455 non-null    int64  
 1   name                   455 non-null    object 
 2   address                455 non-null    object 
 3   coordinates            455 non-null    object 
 4   workday_timing         455 non-null    object 
 5   closed_on              455 non-null    object 
 6   reviews                455 non-null    float64
 7   rating                 455 non-null    float64
 8   most_popular_times     455 non-null    object 
 9   popular_times          455 non-null    object 
 10  reviewer_name          409 non-null    object 
 11  rating_review          409 non-null    object 
 12  review_text            402 non-null    object 
 13  accessibility_enabled  455 non-null    object 
 14  children_enabled       455 non-null    object 
 15  types 

In [30]:
fixedDf.head()

,place_id,name,address,coordinates,workday_timing,closed_on,reviews,rating,most_popular_times,popular_times,reviewer_name,rating_review,review_text,accessibility_enabled,children_enabled,types,city
0,0,Agro Tainment Wisata Kampung Markisa,"Agro Tainment Wisata Kampung Markisa, Blunyahr...","-7.774189,110.3647986",Not Present,Buka Setiap Hari,2.0,5.0,"07.00, 08.00, 09.00\n---\nLabel Waktu: 07.00\n...",Senin:\n 06.00: 0% | Diam\n 07.00: 25% |...,"['Deyze Nghokeh', 'Khoari Koutsar', 'Deyze Ngh...","[5.0, 5.0, 5.0, 5.0]","['Teks review tidak tersedia', 'Teks review ti...","Pintu masuk khusus pengguna kursi roda, Tempat...",Cocok untuk anak-anak,Tourist attraction,"Kota Yogyakarta, Daerah Istimewa Yogyakarta"
1,1,Ancol Wisata Alam,"Ancol Wisata Alam, Unnamed Road, Selingan, Kar...","-7.6668655999999995,110.266713",Not Present,Buka Setiap Hari,390.0,4.4,Tidak Tersedia,Tidak Tersedia,"['Ifandri Dwi Aryono (IFANDRI DWI ARYONO)', 'C...","[5.0, 5.0, 4.0, 5.0, 4.0, 2.0, 5.0, 4.0]",['salah catu calon destinasi wisata alam berik...,"Pintu masuk khusus pengguna kursi roda, Tempat...",Cocok untuk anak-anak,Tourist attraction,"Kabupaten Magelang, Jawa Tengah"
2,2,BANYU WIYOSO DAGEN (Kolam Ikan),"4846+WH5 BANYU WIYOSO DAGEN (Kolam Ikan), Gumu...","-7.8927344999999995,110.3114727",Not Present,Buka Setiap Hari,4.0,5.0,Tidak Tersedia,Tidak Tersedia,"['Laili Isna Fatkhurrahmah', 'Dian Kusuma Dewi...","[5.0, 5.0, 5.0, 5.0]",['Tempat yang asik buat jalan2 pagi atau sore....,"Pintu masuk khusus pengguna kursi roda, Tempat...",Cocok untuk anak-anak,Tourist attraction,"Kabupaten Bantul, Daerah Istimewa Yogyakarta"
3,3,Bendhung Lepen,"Bendhung Lepen, Mrican UH7/ 338, Giwangan, Kec...","-7.8316668,110.3945119",Not Present,Buka Setiap Hari,1336.0,4.7,"17.00, 16.00, 18.00\n---\nLabel Waktu: 17.00\n...",Senin:\n 04.00: 0% | Diam\n 05.00: 6% | ...,"['Muhammad Hidayatullah', 'suwarso doang', 'L ...","[5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0]","['tempat menyenangkan untuk rekreasi, bisa dud...","Pintu masuk khusus pengguna kursi roda, Tempat...","Cocok untuk anak-anak, Cocok untuk ulang tahun...",Tourist attraction,"Kota Yogyakarta, Daerah Istimewa Yogyakarta"
4,4,Bendungan Irigasi Jalan Samas,"278H+G39 Bendungan Irigasi Jalan Samas, Unname...","-7.9837082,110.2777382",Not Present,Buka Setiap Hari,1.0,5.0,Tidak Tersedia,Tidak Tersedia,['Bastian Viery'],[5.0],['Bagus'],Tempat parkir khusus pengguna kursi roda,Tidak cocok untuk anak-anak,Tourist attraction,"Kabupaten Bantul, Daerah Istimewa Yogyakarta"


# Data Preparation

## Standarisasi dan Normalisasi

### Workday Timing

In [31]:
fixedDf.workday_timing.unique()

array(['Not Present', '07.00-11.00', '06.30-04.30', '04.30-05.00',
       '01.00-08.00', '09.00-04.30', '08.00-05.00', 'Open 24 hours',
       '08.00-03.00', '08.00-04.00', '09.00-03.00', '08.30-02.30',
       '08.00-02.00', '04.00-06.00', '08.30-03.30', '09.00-05.00',
       'Closed', '08.00-09.00', '07.00-04.00', '07.30-04.00',
       '11.00-05.00', '04.00-03.00', '06.00-06.00', '08.30-03.00',
       '10.00-10.00', '05.00-04.00', '08.00-06.00', '09.00-09.00',
       '08.30-04.30', '00.00-10.00', '09.00-02.30', '10.00-05.00',
       '09.00-10.00', '06.30-09.00', '06.00-11.00', '07.00-05.00',
       '07.00-06.00', '08.00-08.00', '08.00-06.15', '09.00-04.00',
       '06.00-04.00', '08.17-09.50', '09.30-06.00', '04.00-06.30',
       '07.30-06.30', '05.45-05.00', '09.00-05.30', '07.00-08.00',
       '08.30-09.00', '04.30-09.00', '09.00-08.00', '10.00-03.00',
       '06.00-07.00', '05.00-06.00', '05.00-07.00', '08.00-11.50',
       '09.00-06.00', '07.30-06.00', '08.00-06.30', '03.00-05.00'

## Rating

In [32]:
# Standarisasi Kolom Rating Menggunakan Z-score
scaler = StandardScaler()

# Data 'rating' akan distandarisasi
fixedDf['rating_scaled'] = scaler.fit_transform(fixedDf[['rating']])

## Types

# Model Development

### Perhitungan Haversine (Coordinates)

In [33]:
# Perhitungan jarak menggunakan rumus Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius bumi dalam kilometer
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c  # Hasil dalam kilometer

In [34]:
# 2. Buat dataframe untuk wisata yang mirip secara lokasi
def filter_by_coordinates(df, user_lat, user_lon, max_distance_km=10):
    df['distance'] = df['coordinates'].apply(lambda coord: haversine(user_lat, user_lon, *map(float, coord.split(','))))
    similar_places_df = df[df['distance'] <= max_distance_km]
    
    # Urutkan berdasarkan jarak terdekat
    return similar_places_df.sort_values(by='distance', ascending=True)

### Perhitungan Cosine Similarity

In [35]:
# Modifikasi search_by_name untuk mengurutkan berdasarkan rating dan Cosine Sim
def search_by_name(df, user_input):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['name'])
    user_input_tfidf = tfidf_vectorizer.transform([user_input])

    cosine_sim = cosine_similarity(user_input_tfidf, tfidf_matrix)
    df['similarity'] = cosine_sim[0]
    
    # Urutkan berdasarkan rating setelah similarity
    df = df.sort_values(by=['similarity', 'rating'], ascending=[False, False])
    
    return df

### Filterisasi Waktu Kerja (Working Day)

In [36]:
# Filter berdasarkan waktu operasional (working days)
def filter_by_workday(df, current_time):
    def is_open(workday_timing, current_time):
        if workday_timing == 'Not Present' or workday_timing == 'Closed':
            return False
        try:
            open_time, close_time = workday_timing.split('-')
            open_time = datetime.strptime(open_time.strip(), '%H.%M').time()
            close_time = datetime.strptime(close_time.strip(), '%H.%M').time()
            # Periksa apakah waktu sekarang berada dalam rentang waktu buka
            return open_time <= current_time <= close_time
        except:
            return False

    # Filter berdasarkan apakah tempat wisata sedang buka
    df['is_open'] = df['workday_timing'].apply(lambda x: is_open(x, current_time))
    
    return df[df['is_open'] == True]

### Filterisasi Tipe Wisata

In [37]:
# Filter berdasarkan tipe tempat wisata
def filter_by_type(df, selected_type):
    return df[df['types'].str.contains(selected_type, case=False, na=False)]

## Filterisasi Berdasarkan Jarak Terdekat


In [38]:
# Fungsi untuk mencari tempat wisata terdekat setelah memilih salah satu
def find_nearby_places(df, selected_place_lat, selected_place_lon, max_distance_km=10, top_n=5):
    # Hitung jarak dari tempat wisata yang dipilih
    df['distance_from_selected'] = df['coordinates'].apply(lambda coord: haversine(selected_place_lat, selected_place_lon, *map(float, coord.split(','))))
    
    # Filter tempat wisata yang berada dalam jarak tertentu
    nearby_df = df[df['distance_from_selected'] <= max_distance_km]
    
    # Urutkan berdasarkan jarak terdekat
    nearby_df = nearby_df.sort_values(by='distance_from_selected', ascending=True)
    
    # Kembalikan top-N hasil
    return nearby_df.head(top_n)

# Modifikasi fungsi utama untuk memberikan rekomendasi setelah memilih tempat
def recommend_nearby(df, selected_place_id, max_distance_km=10, top_n=5):
    # Ambil koordinat tempat wisata yang dipilih
    selected_place = df[df['place_id'] == selected_place_id].iloc[0]
    selected_place_lat, selected_place_lon = map(float, selected_place['coordinates'].split(','))

    # Cari tempat wisata terdekat
    nearby_recommendations = find_nearby_places(df, selected_place_lat, selected_place_lon, max_distance_km, top_n)
    
    return nearby_recommendations

## Perhitungan IMDB Weighted Rating

In [39]:
# Fungsi untuk menghitung weighted rating
def weighted_rating(x, m, C):
    v = x['reviews']  # Jumlah ulasan
    R = x['rating']   # Rating rata-rata tempat wisata
    # Hitung weighted rating
    return (v / (v + m) * R) + (m / (v + m) * C)

# 2. Fungsi untuk mengurutkan berdasarkan weighted rating
def sort_by_weighted_rating(df):
    # Tentukan m (jumlah ulasan minimum) dan C (rating rata-rata keseluruhan)
    C = df['rating'].mean()
    m = df['reviews'].quantile(0.75)  # Mengambil persentil 75% dari ulasan untuk threshold minimum

    # Hitung weighted rating untuk setiap baris
    df['weighted_rating'] = df.apply(lambda x: weighted_rating(x, m, C), axis=1)

    # Urutkan berdasarkan weighted rating tertinggi
    return df.sort_values(by='weighted_rating', ascending=False)

## Fungsi Rekomendasi

In [40]:
# Modifikasi fungsi rekomendasi untuk menggunakan weighted rating
def recommend(df, user_input=None, user_lat=None, user_lon=None, selected_type=None, max_distance_km=10, top_n=5):
    # Pencarian berdasarkan nama jika ada input pengguna
    if user_input:
        df = search_by_name(df, user_input)
    
    # Filter berdasarkan tipe tempat wisata jika diberikan
    if selected_type:
        df = filter_by_type(df, selected_type)
    
    # Urutkan berdasarkan weighted rating
    df = sort_by_weighted_rating(df)
    
    # Return hasil pencarian berdasarkan nama dan lokasi
    return df.head(top_n)

## Pencarian Dengan Search Field

In [41]:
# Eksekusi pencarian pertama kali (sebelum memilih tempat)
user_lat = -7.774189
user_lon = 110.3647986
user_input = "Museum"
selected_type = "Museum"
max_distance_km = 0.2
top_n = 20

# Panggil fungsi rekomendasi awal
name_recommendations = recommend(fixedDf, user_input, user_lat, user_lon, selected_type, max_distance_km, top_n)

# Tampilkan hasil
print("Recommendations based on name search:")
name_recommendations

Recommendations based on name search:


,place_id,name,address,coordinates,workday_timing,closed_on,reviews,rating,most_popular_times,popular_times,reviewer_name,rating_review,review_text,accessibility_enabled,children_enabled,types,city,rating_scaled,similarity,weighted_rating
67,67,Museum Sonobudoyo Unit I,"Museum Sonobudoyo Unit I, Jl. Pangurakan No.6,...","-7.802299499999999,110.36438349999999",Not Present,Senin,4860.0,4.8,"11.00, 10.00, 12.00\n---\nLabel Waktu: 11.00\n...",Selasa:\n 06.00: 0% | Diam\n 07.00: 0% |...,"['Frissie Dhitta', 'Reza Anandiraka', 'yurshel...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]",['So far ini museum yang paling edukatif dan ...,"Pintu masuk khusus pengguna kursi roda, Tempat...",Cocok untuk anak-anak,"Tourist attraction, History museum","Kota Yogyakarta, Daerah Istimewa Yogyakarta",0.475239,0.460084,4.739557
209,209,Museum Sandi,"Museum Sandi, Jl. Faridan M Noto No.21, Kotaba...","-7.7845549,110.3711548",09.00-03.00,Tidak Tersedia,1680.0,4.7,Tidak Tersedia,Tidak Tersedia,['Lusiana Lusie'],[5.0],"['Menakjubkan masuk di museum ini, dedikasi ya...",Informasi Tidak Tersedia,Informasi Tidak Tersedia,"Tourist attraction, Museum","Kota Yogyakarta, Daerah Istimewa Yogyakarta",0.389771,0.566030,4.632353
167,167,Museum Wahanarata,"Museum Wahanarata, Jl. Rotowijayan, Kadipaten,...","-7.805466999999999,110.363062",09.00-03.00,Tidak Tersedia,1163.0,4.7,Tidak Tersedia,Tidak Tersedia,['Alno Channel'],[5.0],['Cocok untuk momong anak atau membahagiakan i...,Informasi Tidak Tersedia,Informasi Tidak Tersedia,"Tourist attraction, History museum","Kota Yogyakarta, Daerah Istimewa Yogyakarta",0.389771,0.566030,4.620051
205,205,"Sonobudoyo Museum, Unit II","Sonobudoyo Museum, Unit II, Jl. Wijilan No.27D...","-7.804463399999999,110.3661027",07.30-04.00,Tidak Tersedia,334.0,4.8,Tidak Tersedia,Tidak Tersedia,['Faizhal Arif'],[3.0],['Museum milik pribadi sepertinya dan tempatny...,Informasi Tidak Tersedia,Informasi Tidak Tersedia,"Tourist attraction, Museum","Kota Yogyakarta, Daerah Istimewa Yogyakarta",0.475239,0.382209,4.603727
194,194,Vredeburg Museum,"Vredeburg Museum, Jl. Margo Mulyo No.6, Ngupas...","-7.800293,110.3661642",Not Present,Buka Setiap Hari,18005.0,4.6,Tidak Tersedia,Tidak Tersedia,['Dony Cavalera'],[4.0],['Seru banget.... Salah trip yang luar biasa m...,Informasi Tidak Tersedia,Informasi Tidak Tersedia,"Tourist attraction, Fortress, Natural history ...","Kota Yogyakarta, Daerah Istimewa Yogyakarta",0.304303,0.566030,4.596988
450,450,Chocolate Monggo Museum New Factory Store Keda...,Chocolate Monggo Museum New Factory Store Keda...,"-7.8406888,110.2978189",09.00-05.00,Tidak Tersedia,2866.0,4.6,Tidak Tersedia,Tidak Tersedia,NaN,NaN,NaN,Informasi Tidak Tersedia,Informasi Tidak Tersedia,"Tourist attraction, Cafe, Chocolate factory, C...","Kabupaten Bantul, Daerah Istimewa Yogyakarta",0.304303,0.257589,4.586957
192,192,Museum Sri Sultan Hamengkubuwono IX,"Museum Sri Sultan Hamengkubuwono IX, Panembaha...","-7.8083193,110.3644395",08.30-03.00,Tidak Tersedia,307.0,4.7,Tidak Tersedia,Tidak Tersedia,['Zura Novianti'],[5.0],"['Ke sini mau beli wingko, apem beras, dan car...",Informasi Tidak Tersedia,Informasi Tidak Tersedia,"Tourist attraction, Museum","Kota Yogyakarta, Daerah Istimewa Yogyakarta",0.389771,0.333954,4.585609
66,66,Museum Rumah Garuda,"38RR+9PF Museum Rumah Garuda, Bantul, Area Saw...","-7.909063499999999,110.341792",Not Present,Buka Setiap Hari,186.0,4.7,"19.00, 18.00, 20.00\n---\nLabel Waktu: 19.00\n...",Senin:\n 06.00: 0% | Diam\n 07.00: 0% | ...,"['Lia Jap', 'Dandy Dzakyesa', 'Rey Tanjung', '...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]",['Senang bisa menemani akang menemui Pakde Nan...,"Pintu masuk khusus pengguna kursi roda, Tempat...",Cocok untuk anak-anak,"Tourist attraction, History museum","Kabupaten Bantul, Daerah Istimewa Yogyakarta",0.389771,0.462207,4.578192
345,345,Tourism Village Mangir,"37VG+6MR Tourism Village Mangir, Mangir Kidul,...","-7.9068983,110.2766362",Open 24 hours,Tidak Tersedia,61.0,4.7,Tidak Tersedia,Tida

## Pencarian dari Tempat Yang Dipilih

In [42]:
# Eksekusi pencarian berdasarkan tempat yang dipilih
selected_place_id = 67  # Misalnya, pengguna memilih tempat dengan place_id 0
max_distance_km = 5  # Jarak maksimal dari tempat yang dipilih
top_n = 10  # Ambil 3 tempat terdekat

# Periksa apakah rekomendasi pertama berisi kolom place_id
if 'place_id' in name_recommendations.columns:
    # Panggil fungsi rekomendasi untuk tempat terdekat
    nearby_recommendations = recommend_nearby(name_recommendations, selected_place_id, max_distance_km, top_n)

    # Tampilkan hasil rekomendasi terdekat menggunakan display()
    print("\nNearby recommendations from the selected place:")
    display(nearby_recommendations)  # Menggunakan display untuk menampilkan tabel dengan lebih baik
else:
    print("Place ID tidak ditemukan dalam hasil pencarian.")


Nearby recommendations from the selected place:


,place_id,name,address,coordinates,workday_timing,closed_on,reviews,rating,most_popular_times,popular_times,...,rating_review,review_text,accessibility_enabled,children_enabled,types,city,rating_scaled,similarity,weighted_rating,distance_from_selected
67,67,Museum Sonobudoyo Unit I,"Museum Sonobudoyo Unit I, Jl. Pangurakan No.6,...","-7.802299499999999,110.36438349999999",Not Present,Senin,4860.0,4.8,"11.00, 10.00, 12.00\n---\nLabel Waktu: 11.00\n...",Selasa:\n 06.00: 0% | Diam\n 07.00: 0% |...,...,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]",['So far ini museum yang paling edukatif dan ...,"Pintu masuk khusus pengguna kursi roda, Tempat...",Cocok untuk anak-anak,"Tourist attraction, History museum","Kota Yogyakarta, Daerah Istimewa Yogyakarta",0.475239,0.460084,4.739557,0.000000
194,194,Vredeburg Museum,"Vredeburg Museum, Jl. Margo Mulyo No.6, Ngupas...","-7.800293,110.3661642",Not Present,Buka Setiap Hari,18005.0,4.6,Tidak Tersedia,Tidak Tersedia,...,[4.0],['Seru banget.... Salah trip yang luar biasa m...,Informasi Tidak Tersedia,Informasi Tidak Tersedia,"Tourist attraction, Fortress, Natural history ...","Kota Yogyakarta, Daerah Istimewa Yogyakarta",0.304303,0.566030,4.596988,0.297091
205,205,"Sonobudoyo Museum, Unit II","Sonobudoyo Museum, Unit II, Jl. Wijilan No.27D...","-7.804463399999999,110.3661027",07.30-04.00,Tidak Tersedia,334.0,4.8,Tidak Tersedia,Tidak Tersedia,...,[3.0],['Museum milik pribadi sepertinya dan tempatny...,Informasi Tidak Tersedia,Informasi Tidak Tersedia,"Tourist attraction, Museum","Kota Yogyakarta, Daerah Istimewa Yogyakarta",0.475239,0.382209,4.603727,0.306213
167,167,Museum Wahanarata,"Museum Wahanarata, Jl. Rotowijayan, Kadipaten,...","-7.805466999999999,110.363062",09.00-03.00,Tidak Tersedia,1163.0,4.7,Tidak Tersedia,Tidak Tersedia,...,[5.0],['Cocok untuk momong anak atau membahagiakan i...,Informasi Tidak Tersedia,Informasi Tidak Tersedia,"Tourist attraction, History museum","Kota Yogyakarta, Daerah Istimewa Yogyakarta",0.389771,0.566030,4.620051,0.381112
192,192,Museum Sri Sultan Hamengkubuwono IX,"Museum Sri Sultan Hamengkubuwono IX, Panembaha...","-7.8083193,110.3644395",08.30-03.00,Tidak Tersedia,307.0,4.7,Tidak Tersedia,Tidak Tersedia,...,[5.0],"['Ke sini mau beli wingko, apem beras, dan car...",Informasi Tidak Tersedia,Informasi Tidak Tersedia,"Tourist attraction, Museum","Kota Yogyakarta, Daerah Istimewa Yogyakarta",0.389771,0.333954,4.585609,0.669400
168,168,Maritime Museum - Jogja,"Maritime Museum - Jogja, Jl. R. E. Martadinata...","-7.801140599999999,110.3493008",08.30-03.30,Tidak Tersedia,225.0,4.4,Tidak Tersedia,Tidak Tersedia,...,NaN,NaN,Informasi Tidak Tersedia,Informasi Tidak Tersedia,"Tourist attraction, Museum","Kabupaten Bantul, Daerah Istimewa Yogyakarta",0.133368,0.501597,4.545252,1.666585
220,220,Cemeti - Institute for Art and Society,"Cemeti - Institute for Art and Society, Jl. DI...","-7.8176311,110.362438",10.00-05.00,Tidak Tersedia,222.0,4.6,Tidak Tersedia,Tidak Tersedia,...,[4.0],"['Embung nya ada di dalem kampung, tidak ada p...",Informasi Tidak Tersedia,Informasi Tidak Tersedia,"Tourist attraction, Art gallery, Arts organiza...","Kota Yogyakarta, Daerah Istimewa Yogyakarta",0.304303,0.000000,4.568825,1.718215
211,211,Sekretariat Sementara Museum Pendidikan Dan Ma...,Sekretariat Sementara Museum Pendidikan Dan Ma...,"-7.8187235,110.3647195",Not Present,Buka Setiap Hari,60.0,4.3,Tidak Tersedia,Tidak Tersedia,...,NaN,NaN,Informasi Tidak Tersedia,Informasi Tidak Tersedia,"Tourist attraction, Museum, Toy museum","Kota Yogyakarta, Daerah Istimewa Yogyakarta",0.047900,0.266766,4.555578,1.826641
209,209,Museum Sandi,"Museum Sandi, Jl. Faridan M Noto No.21, Kotaba...","-7.7845549,110.3711548",09.00-03.00,Tidak Tersedia,1680.0,4.7,Tidak Tersedia,Tidak Tersedia,...,[5.0],"['Menakjubkan masuk di museum ini, dedikasi ya...",Informasi Tidak Tersedia,Informasi Tidak Tersedia,"Tourist attraction, Museum","Kota Yogyakarta, Daerah Istimewa Yogyakarta",0.389771,0.566030,4.632353,2.10941

## Model Evaluation

In [46]:
from sklearn.metrics import mean_absolute_error

# Assuming 'rating' is the actual value and 'weighted_rating' is the predicted value
def calculate_mae(df):
    # Ensure that 'weighted_rating' is already computed
    if 'weighted_rating' not in df.columns:
        # Calculate 'weighted_rating' if not present
        C = df['rating'].mean()
        m = df['reviews'].quantile(0.75)
        df['weighted_rating'] = df.apply(lambda x: weighted_rating(x, m, C), axis=1)
    
    # Calculate MAE between actual 'rating' and 'weighted_rating'
    mae_value = mean_absolute_error(df['rating'], df['weighted_rating'])
    
    return mae_value

# Calculate the MAE for the dataset
mae = calculate_mae(fixedDf)

print("Mean Absolute Error (MAE) between rating and weighted rating:", mae)


Mean Absolute Error (MAE) between rating and weighted rating: 0.5442252761302397


## Coba

In [ ]:
# Contoh dataset riwayat rating user
# Kolom: user_id, place_id, rating
user_ratings = pd.DataFrame({
    'user_id': [1, 1, 1, 2, 2],
    'place_id': [10, 20, 30, 10, 40],
    'rating': [4, 5, 3, 5, 4]
})

# Step 1: Memeriksa tempat yang pernah dirating oleh user
def get_user_rated_places(user_id, ratings_df):
    user_rated_places = ratings_df[ratings_df['user_id'] == user_id]
    return user_rated_places

# Step 2: Menghitung kemiripan antar tempat berdasarkan rating
def calculate_similarity_based_on_ratings(place_id, ratings_df):
    # Membuat pivot table: baris = place, kolom = user, nilai = rating
    rating_pivot = ratings_df.pivot_table(index='place_id', columns='user_id', values='rating').fillna(0)
    
    # Menghitung cosine similarity antar tempat
    similarity_matrix = cosine_similarity(rating_pivot)
    
    # Menyusun hasil dalam dataframe
    similarity_df = pd.DataFrame(similarity_matrix, index=rating_pivot.index, columns=rating_pivot.index)
    
    # Mengurutkan tempat yang mirip dengan place_id yang dipilih
    similar_places = similarity_df[place_id].sort_values(ascending=False)
    return similar_places

# Step 3: Menggabungkan hasil rekomendasi awal dengan hasil dari rating user
def combine_recommendations(initial_recommendations, user_rated_places, ratings_df):
    combined_results = pd.DataFrame()
    
    for place_id in user_rated_places['place_id']:
        # Dapatkan tempat yang mirip berdasarkan rating user
        similar_places = calculate_similarity_based_on_ratings(place_id, ratings_df)
        
        # Gabungkan dengan hasil rekomendasi awal
        combined_results = pd.concat([combined_results, initial_recommendations[initial_recommendations['place_id'].isin(similar_places.index)]])
    
    # Menghapus duplikat dan mengurutkan berdasarkan relevansi
    combined_results = combined_results.drop_duplicates().sort_values(by='rating', ascending=False)
    
    return combined_results

# Step 4: Menjalankan kombinasi rekomendasi
user_id = 1  # Contoh user
user_rated_places = get_user_rated_places(user_id, user_ratings)
final_recommendations = combine_recommendations(name_recommendations, user_rated_places, user_ratings)

# Menampilkan hasil rekomendasi final
display(final_recommendations)
